In [5]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap
import hashlib
import numpy as np

power_plants_df = pd.read_csv('global_power_plant_database.csv', low_memory=False)

power_plants_df.rename(columns={
    'country_long': 'country_full_name',
    'country': 'country_code',
    'latitude': 'lat',
    'longitude': 'lng',
    'fuel1': 'fuel_type',
    'capacity_mw': 'capacity',
    'commissioning_year': 'commission_year',
}, inplace=True)

power_plants_df.dropna(subset=['lat', 'lng'], inplace=True)

power_plants_df['capacity'] = pd.to_numeric(power_plants_df['capacity'], errors='coerce').fillna(0)
power_plants_df['commission_year'] = pd.to_numeric(power_plants_df['commission_year'], errors='coerce').fillna(0).astype(int)

generation_cols = [col for col in power_plants_df.columns if 'generation_gwh_' in col]

for col in generation_cols:
    power_plants_df[col] = pd.to_numeric(power_plants_df[col], errors='coerce').fillna(0)

if 'generation_gwh_2016' in power_plants_df.columns:
    power_plants_df['latest_generation_gwh'] = power_plants_df['generation_gwh_2016']
elif 'generation_gwh_2017' in power_plants_df.columns:
     power_plants_df['latest_generation_gwh'] = power_plants_df['generation_gwh_2017']
else:
    if generation_cols:
        power_plants_df['latest_generation_gwh'] = power_plants_df[generation_cols].sum(axis=1)
    else:
        power_plants_df['latest_generation_gwh'] = 0

power_plants_df = power_plants_df[power_plants_df['capacity'] > 0].copy()

fuel_colors = {
    'Hydro': '#4682B4',
    'Solar': '#FFD700',
    'Wind': '#00BFFF',
    'Gas': '#FFA500',
    'Coal': '#696969',
    'Nuclear': '#FF6347',
    'Oil': '#8B4513',
    'Biomass': '#228B22',
    'Geothermal': '#8A2BE2',
    'Waste': '#D2B48C',
    'Other': '#A9A9A9',
    'Cogeneration': '#DDA0DD',
    'Storage': '#CD853F',
    'Wave and Tidal': '#40E0D0'
}
power_plants_df['color'] = power_plants_df['fuel_type'].apply(lambda x: fuel_colors.get(x, '#A9A9A9'))

m3 = folium.Map(location=[20, 0], zoom_start=2)

heat_data = [[row['lat'], row['lng'], row['capacity'] / 1000]
             for index, row in power_plants_df.sample(n=min(20000, len(power_plants_df)), random_state=42).iterrows() if row['capacity'] > 0]
if heat_data:
    HeatMap(heat_data, radius=10).add_to(m3)

marker_cluster_plants = MarkerCluster().add_to(m3)

sampled_plants_for_markers = power_plants_df.nlargest(min(5000, len(power_plants_df)), 'capacity', keep='first')

for idx, row in sampled_plants_for_markers.iterrows():
    generation_info = ""
    for col in generation_cols:
        if row[col] > 0:
            year = col.replace('generation_gwh_', '')
            generation_info += f"<p>Generáció {year}: <b>{row[col]:,.0f} GWh</b></p>"

    owner_info = f"<p>Tulajdonos: {row['owner'] if pd.notna(row['owner']) else 'Nincs adat'}</p>"
    url_info = f"<p>Forrás URL: <a href='{row['url']}' target='_blank'>Link</a></p>" if pd.notna(row['url']) else ""

    popup_html = f"""
    <h4>{row['name'] if pd.notna(row['name']) else 'Ismeretlen Erőmű'}</h4>
    <p>Ország: <b>{row['country_full_name']} ({row['country_code']})</b></p>
    <p>Üzemanyag típus: <b>{row['fuel_type'] if pd.notna(row['fuel_type']) else 'Nincs adat'}</b></p>
    <p>Kapacitás: <b>{row['capacity']:,} MW</b></p>
    <p>Üzembe helyezés éve: <b>{row['commission_year'] if row['commission_year'] > 0 else 'Nincs adat'}</b></p>
    {generation_info}
    {owner_info}
    {url_info}
    <p>Adat forrás: {row['source'] if pd.notna(row['source']) else 'Nincs adat'}</p>
    <p>Koordináták: {row['lat']:.2f}, {row['lng']:.2f}</p>
    """
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=3,
        color=row['color'],
        fill=True,
        fill_color=row['color'],
        fill_opacity=0.7,
        tooltip=f"{row['name'] if pd.notna(row['name']) else 'Ismeretlen'} ({row['fuel_type']})",
        popup=folium.Popup(popup_html, max_width=400)
    ).add_to(marker_cluster_plants)

total_plants = len(power_plants_df)
total_capacity = power_plants_df['capacity'].sum()
average_capacity = power_plants_df['capacity'].mean()
num_countries_with_plants = power_plants_df['country_full_name'].nunique()
num_fuel_types = power_plants_df['fuel_type'].nunique()

top_10_countries_by_plants = power_plants_df['country_full_name'].value_counts().head(10)
top_10_countries_by_capacity = power_plants_df.groupby('country_full_name')['capacity'].sum().nlargest(10)
top_5_fuel_types_by_capacity = power_plants_df.groupby('fuel_type')['capacity'].sum().nlargest(5)
top_5_fuel_types_by_count = power_plants_df['fuel_type'].value_counts().head(5)

largest_plant = power_plants_df.loc[power_plants_df['capacity'].idxmax()]

stats_html_content = f"""
    <h3>Globális Erőmű Statisztikák</h3>
    <p>Összes erőmű (adatbázisban): <b>{total_plants:,}</b></p>
    <p>Összes kapacitás: <b>{total_capacity:,.0f} MW</b></p>
    <p>Átlagos kapacitás (erőművenként): <b>{average_capacity:,.0f} MW</b></p>
    <p>Képviselt országok száma: <b>{num_countries_with_plants}</b></p>
    <p>Különböző üzemanyag típusok: <b>{num_fuel_types}</b></p>
    <h4>Top 10 ország (erőművek száma alapján)</h4>
    <ul>
"""
for country, count in top_10_countries_by_plants.items():
    stats_html_content += f"<li>{country}: {count} erőmű</li>"
stats_html_content += "</ul>"

stats_html_content += """
    <h4>Top 10 ország (összes kapacitás alapján)</h4>
    <ul>
"""
for country, capacity in top_10_countries_by_capacity.items():
    stats_html_content += f"<li>{country}: {capacity:,.0f} MW</li>"
stats_html_content += "</ul>"

stats_html_content += """
    <h4>Top 5 üzemanyag típus (kapacitás alapján)</h4>
    <ul>
"""
for fuel, capacity in top_5_fuel_types_by_capacity.items():
    stats_html_content += f"<li>{fuel}: {capacity:,.0f} MW</li>"
stats_html_content += "</ul>"

stats_html_content += """
    <h4>Top 5 üzemanyag típus (erőművek száma alapján)</h4>
    <ul>
"""
for fuel, count in top_5_fuel_types_by_count.items():
    stats_html_content += f"<li>{fuel}: {count} erőmű</li>"
stats_html_content += "</ul>"

stats_html_content += f"""
    <h4>Legnagyobb erőmű (kapacitás alapján)</h4>
    <ul>
        <li>Név: <b>{largest_plant['name'] if pd.notna(largest_plant['name']) else 'Ismeretlen'}</b></li>
        <li>Ország: {largest_plant['country_full_name']}</li>
        <li>Üzemanyag: {largest_plant['fuel_type'] if pd.notna(largest_plant['fuel_type']) else 'Nincs adat'}</li>
        <li>Kapacitás: {largest_plant['capacity']:,} MW</li>
    </ul>
"""

stats_box_html = f"""
    <div style="position: fixed;
                bottom: 20px; left: 20px; width: 300px; height: auto;
                z-index:9999; font-size:14px;
                background-color:white;
                opacity:0.9;
                padding:15px;
                border-radius:8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);
                max-height: 550px;
                overflow-y: auto;">
        {stats_html_content}
    </div>
"""

m3.get_root().html.add_child(folium.Element(stats_box_html))

m3.save("globalis_eromu_terkep.html")

In [6]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

csv_file = "whc-sites(tangibles)-2021.csv"

try:
    df = pd.read_csv(csv_file)
except FileNotFoundError:
    exit()
except pd.errors.EmptyDataError:
    exit()
except Exception as e:
    exit()

df.rename(columns={
    'Name': 'name',
    'short_description': 'short_description',
    'date_inscribed': 'date_inscribed',
    'longitude': 'longitude',
    'latitude': 'latitude',
    'area_hectares': 'area_hectares',
    'category_long': 'category',
    'category_short': 'category_short',
    'Country name': 'country_name',
    'Region': 'region',
    'iso_code': 'iso_code'
}, inplace=True)

df['short_description'] = df['short_description'].astype(str).str.replace(r'<[^>]*>', '', regex=True)
df['name'] = df['name'].astype(str).str.replace(r'<[^>]*>', '', regex=True)

df['date_inscribed'] = pd.to_numeric(df['date_inscribed'], errors='coerce')
df['area_hectares'] = pd.to_numeric(df['area_hectares'], errors='coerce')

total_sites = len(df)
cultural_sites = df[df['category'] == 'Cultural'].shape[0]
natural_sites = df[df['category'] == 'Natural'].shape[0]
mixed_sites = df[df['category'] == 'Mixed'].shape[0]

earliest_year = int(df['date_inscribed'].min()) if pd.notna(df['date_inscribed'].min()) else 'N/A'
latest_year = int(df['date_inscribed'].max()) if pd.notna(df['date_inscribed'].max()) else 'N/A'

avg_area = df['area_hectares'].mean()
max_area_site = df.loc[df['area_hectares'].idxmax()] if not df['area_hectares'].isnull().all() else None

top_countries = df['country_name'].value_counts().head(5)

m = folium.Map(location=[48.8566, 2.3522], zoom_start=3, tiles="CartoDB positron")

marker_cluster = MarkerCluster().add_to(m)

def get_icon_details(category):
    if category == 'Cultural':
        return 'landmark', 'blue'
    elif category == 'Natural':
        return 'tree', 'green'
    elif category == 'Mixed':
        return 'star', 'purple'
    else:
        return 'question', 'gray'

for idx, row in df.iterrows():
    try:
        name = row['name']
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        category = row['category']
        country = row['country_name']
        date_inscribed = row['date_inscribed']
        short_desc = row['short_description']
        area = row['area_hectares']

        url = "https://whc.unesco.org/en/list/"

        popup_html = f"""
        <h4>{name}</h4>
        <p>
        <b>Ország:</b> {country}<br>
        <b>Kategória:</b> {category}<br>
        <b>Felvétel éve:</b> {int(date_inscribed) if pd.notna(date_inscribed) else 'N/A'}<br>
        <b>Terület (ha):</b> {f'{area:,.2f}' if pd.notna(area) else 'N/A'}<br>
        <b>Rövid leírás:</b> {short_desc[:300] + '...' if len(str(short_desc)) > 300 else short_desc}<br>
        <a href="{url}" target="_blank">További részletek (UNESCO)</a>
        </p>
        """
        iframe = folium.IFrame(popup_html, width=300, height=200)
        popup = folium.Popup(iframe, max_width=300)

        tooltip_text = f"{name} ({category})"

        icon_symbol, icon_color = get_icon_details(category)

        if pd.notna(lat) and pd.notna(lon) and -90 <= lat <= 90 and -180 <= lon <= 180:
            folium.Marker(
                location=[lat, lon],
                popup=popup,
                tooltip=tooltip_text,
                icon=folium.Icon(color=icon_color, icon=icon_symbol, prefix='fa')
            ).add_to(marker_cluster)
    except (ValueError, TypeError, KeyError) as e:
        pass

statistics_html = f"""
    <div style="position: fixed;
                bottom: 20px; left: 20px; width: 300px; height: auto;
                z-index:9999; font-size:14px;
                background-color:white;
                opacity:0.9;
                padding:15px;
                border-radius:8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);">
        <h4>Világörökségi Helyszínek Statisztikái</h4>
        <p><b>Összes helyszín:</b> {total_sites}</p>
        <p><b>Kategóriák:</b></p>
        <ul>
            <li>Kulturális: {cultural_sites}</li>
            <li>Természeti: {natural_sites}</li>
            <li>Vegyes: {mixed_sites}</li>
        </ul>
        <p><b>Felvétel évei:</b></p>
        <ul>
            <li>Legkorábbi: {earliest_year}</li>
            <li>Legkésőbbi: {latest_year}</li>
        </ul>
        <p><b>Terület:</b></p>
        <ul>
            <li>Átlagos terület: {f'{avg_area:,.2f}' if pd.notna(avg_area) else 'N/A'} ha</li>
            <li>Legnagyobb helyszín: {max_area_site['name'] if max_area_site is not None else 'N/A'} ({f'{max_area_site["area_hectares"]:,.2f}' if max_area_site is not None else 'N/A'} ha)</li>
        </ul>
        <p><b>Top 5 ország:</b></p>
        <ol>
            {"".join([f"<li>{country}: {count}</li>" for country, count in top_countries.items()])}
        </ol>
    </div>
"""

m.get_root().html.add_child(folium.Element(statistics_html))

map_filename = "unesco_vilagorokseg_terkep_uj_adatforras.html"
m.save(map_filename)

In [7]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap
import hashlib

cities_df = pd.read_csv('worldcitiespop.csv', low_memory=False)

cities_df.rename(columns={
    'Country': 'country',
    'City': 'city',
    'AccentCity': 'accent_city',
    'Region': 'region',
    'Population': 'population',
    'Latitude': 'lat',
    'Longitude': 'lng'
}, inplace=True)

cities_df['population'] = pd.to_numeric(cities_df['population'], errors='coerce').fillna(0)

cities_df['color'] = cities_df['country'].apply(lambda x: '#' + hashlib.md5(str(x).encode()).hexdigest()[:6])

m2 = folium.Map(location=[20, 0], zoom_start=2)

heat_data = [[row['lat'], row['lng'], row['population'] / 1000000]
             for index, row in cities_df[cities_df['population'] > 0].sample(n=min(10000, len(cities_df[cities_df['population'] > 0])), random_state=42).iterrows()]
HeatMap(heat_data, radius=15).add_to(m2)

marker_cluster_all_cities = MarkerCluster().add_to(m2)

sampled_cities_for_markers = cities_df[cities_df['population'] >= 100000].sample(n=min(5000, len(cities_df[cities_df['population'] >= 100000])), random_state=42)

for idx, row in sampled_cities_for_markers.iterrows():
    popup_html = f"""
    <h4>{row['city']} ({row['country'].upper()})</h4>
    <p>Népesség: <b>{row['population']:,}</b> fő</p>
    <p>Ország: {row['country']}</p>
    <p>Adminisztratív terület: {row['region'] if pd.notna(row['region']) else 'Nincs adat'}</p>
    <p>Koordináták: {row['lat']:.2f}, {row['lng']:.2f}</p>
    """
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=5,
        color=row['color'],
        fill=True,
        fill_color=row['color'],
        fill_opacity=0.7,
        tooltip=f"{row['city']}, {row['country'].upper()}",
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(marker_cluster_all_cities)

total_population_sampled_markers = sampled_cities_for_markers['population'].sum()
average_population_sampled_markers = sampled_cities_for_markers['population'].mean()
median_population_sampled_markers = sampled_cities_for_markers['population'].median()
num_countries_represented = sampled_cities_for_markers['country'].nunique()

top_10_countries_by_cities = sampled_cities_for_markers['country'].value_counts().head(10)
top_10_countries_by_population = sampled_cities_for_markers.groupby('country')['population'].sum().nlargest(10)

cities_with_over_1_million_population = sampled_cities_for_markers[sampled_cities_for_markers['population'] >= 1000000]
num_cities_over_1_million = len(cities_with_over_1_million_population)
total_population_over_1_million = cities_with_over_1_million_population['population'].sum()
average_population_over_1_million = cities_with_over_1_million_population['population'].mean() if num_cities_over_1_million > 0 else 0

cities_with_over_5_million_population = sampled_cities_for_markers[sampled_cities_for_markers['population'] >= 5000000]
num_cities_over_5_million = len(cities_with_over_5_million_population)
total_population_over_5_million = cities_with_over_5_million_population['population'].sum()

most_populous_city = sampled_cities_for_markers.loc[sampled_cities_for_markers['population'].idxmax()]
least_populous_city = sampled_cities_for_markers.loc[sampled_cities_for_markers['population'].idxmin()]

stats_html_content = f"""
    <h3>Térkép Statisztikák</h3>
    <p>Jelölt városok száma: <b>{len(sampled_cities_for_markers):,}</b></p>
    <p>Jelölt városok teljes népessége: <b>{total_population_sampled_markers:,}</b> fő</p>
    <p>Jelölt városok átlagos népessége: <b>{average_population_sampled_markers:,.0f}</b> fő</p>
    <p>Jelölt városok medián népessége: <b>{median_population_sampled_markers:,.0f}</b> fő</p>
    <p>Képviselt országok száma: <b>{num_countries_represented}</b></p>
    <p>1 millió főnél nagyobb városok száma: <b>{num_cities_over_1_million}</b></p>
    <p>1 millió főnél nagyobb városok teljes népessége: <b>{total_population_over_1_million:,}</b> fő</p>
    <p>1 millió főnél nagyobb városok átlagos népessége: <b>{average_population_over_1_million:,.0f}</b> fő</p>
    <p>5 millió főnél nagyobb városok száma: <b>{num_cities_over_5_million}</b></p>
    <p>5 millió főnél nagyobb városok teljes népessége: <b>{total_population_over_5_million:,}</b> fő</p>
    <h4>Extrém értékek a jelölt városok között:</h4>
    <ul>
        <li>Legnépesebb város: <b>{most_populous_city['city']}</b> ({most_populous_city['country']}) - {most_populous_city['population']:,} fő</li>
        <li>Legkevésbé népes város: <b>{least_populous_city['city']}</b> ({least_populous_city['country']}) - {least_populous_city['population']:,} fő</li>
    </ul>
    <h4>Top 10 ország (városok száma alapján)</h4>
    <ul>
"""
for country, count in top_10_countries_by_cities.items():
    stats_html_content += f"<li>{country}: {count} város</li>"
stats_html_content += "</ul>"

stats_html_content += """
    <h4>Top 10 ország (teljes népesség alapján)</h4>
    <ul>
"""
for country, population in top_10_countries_by_population.items():
    stats_html_content += f"<li>{country}: {population:,.0f} fő</li>"
stats_html_content += "</ul>"

stats_box_html = f"""
    <div style="position: fixed;
                bottom: 20px; left: 20px; width: 300px; height: auto;
                z-index:9999; font-size:14px;
                background-color:white;
                opacity:0.9;
                padding:15px;
                border-radius:8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.2);
                max-height: 450px;
                overflow-y: auto;">
        {stats_html_content}
    </div>
"""

m2.get_root().html.add_child(folium.Element(stats_box_html))

m2.save("vilag_varosai_heatmap_es_szinezes.html")

In [8]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap

try:
    cities_df = pd.read_csv('worldcitiespop.csv', low_memory=False)
    print("Az adathalmaz sikeresen betöltve.")
    print(cities_df.head())
except FileNotFoundError:
    print("Hiba: A 'worldcitiespop.csv' fájl nem található.")

cities_df.rename(columns={
    'Country': 'country',
    'City': 'city',
    'Population': 'population',
    'Latitude': 'lat',
    'Longitude': 'lng'
}, inplace=True)

cities_df['population'] = pd.to_numeric(cities_df['population'], errors='coerce').fillna(0).astype(int)

cities_df.dropna(subset=['lat', 'lng'], inplace=True)

top_cities = cities_df.sort_values(by='population', ascending=False).head(5000).copy()

max_global_population = cities_df['population'].max()

m1 = folium.Map(location=[20, 0], zoom_start=2)

marker_cluster = MarkerCluster().add_to(m1)

for idx, row in top_cities.iterrows():
    radius = max(3, row['population'] / 1000000)

    normalized_population = row['population'] / max_global_population if max_global_population > 0 else 0
    color = f"hsl({120 * (1 - normalized_population)}, 100%, 40%)"

    thermometer_height = normalized_population * 100
    popup_html = f"""
    <h4>{row['city']} ({row['country'].upper()})</h4>
    <p>Népesség: <b>{row['population']:,}</b> fő</p>
    <div style="width: 100px; height: 120px; border: 1px solid #ccc; background-color: #f0f0f0; margin: 10px auto; display: flex; flex-direction: column-reverse; position: relative;">
        <div style="width: 100%; height: {thermometer_height:.1f}%; background-color: {color};"></div>
        <div style="position: absolute; top: 0; left: 0; width: 100%; text-align: center; font-size: 10px; color: black; z-index: 1;">
            {thermometer_height:.1f}%
        </div>
    </div>
    """

    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        tooltip=f"{row['city']}, {row['country'].upper()}",
        popup=folium.Popup(popup_html, max_width=250)
    ).add_to(marker_cluster)

m1.save("vilag_legnepesebb_varosai.html")
print("A 'vilag_legnepesebb_varosai.html' térkép elkészült.")

Az adathalmaz sikeresen betöltve.
  Country        City  AccentCity Region  Population   Latitude  Longitude
0      ad       aixas       Aixàs     06         NaN  42.483333   1.466667
1      ad  aixirivali  Aixirivali     06         NaN  42.466667   1.500000
2      ad  aixirivall  Aixirivall     06         NaN  42.466667   1.500000
3      ad   aixirvall   Aixirvall     06         NaN  42.466667   1.500000
4      ad    aixovall    Aixovall     06         NaN  42.466667   1.483333
A 'vilag_legnepesebb_varosai.html' térkép elkészült.
